In [1]:
import copy
import json
import os
import time
from datetime import datetime
from pprint import pprint
import redis
import requests
from celery import Celery, Task, platforms
from openpyxl import Workbook
from openpyxl.styles import Alignment, Border, Font, PatternFill, Side, colors
from openpyxl.utils import column_index_from_string, get_column_letter
from pymongo import MongoClient

import conf

class ReportTask():
    abstract = True
    _rdb1 = None
    _rdb2 = None
    _mdb = None

    @property
    def rdb_1(self):
        if self._rdb1 is None:
            self._rdb1 = redis.StrictRedis(host=conf.REDIS_PAYMENT_HOST,
                                           port=conf.REDIS_ACTIVITY_PORT,
                                           db=conf.REDIS_ACTIVITY_DB_1,
                                           charset="utf-8",
                                           decode_responses=True)
        return self._rdb1

    @property
    def rdb_2(self):
        if self._rdb2 is None:
            self._rdb2 = redis.StrictRedis(host=conf.REDIS_PAYMENT_HOST,
                                           port=conf.REDIS_ACTIVITY_PORT,
                                           db=conf.REDIS_ACTIVITY_DB_2,
                                           charset="utf-8",
                                           decode_responses=True)
        return self._rdb2

    @property
    def mdb(self):
        if self._mdb is None:
            _mc = MongoClient(conf.MONGO_HOST, replicaset=conf.REPLICASET_NAME)
            self._mdb = _mc['boluome']
        return self._mdb

r=ReportTask()

In [3]:
def generate_transaction_report(reports_data, field_headers, out_path):
    wb = Workbook()
    ws = wb.active
    ws.append(field_headers)

    def render_report(ws):
        # 冻结第一行
        ws.freeze_panes = 'A2'
        for i in range(1, ws.max_column + 1):
            # 抬头第一行 设置填充色、字体格式、居中
            ws['{}1'.format(get_column_letter(i))].fill = PatternFill("solid", fgColor="FEC000")
            ws['{}1'.format(get_column_letter(i))].font = Font(name="微软雅黑", size=12, bold=True)
            ws['{}1'.format(get_column_letter(i))].alignment = Alignment(horizontal='center', vertical='center')

        # 为 所有表格设置边框和字体。
        left, right, top, bottom = [Side(style='thin', color='000000')] * 4
        for row in range(1, ws.max_row + 1):
            for col in range(1, ws.max_column + 1):
                ws.cell(row=row, column=col).border = Border(
                    left=left, right=right, top=top, bottom=bottom)  # 设置单元格边框格式

        for row in range(2, ws.max_row + 1):
            for col in range(1, ws.max_column + 1):
                ws.cell(row=row, column=col).font = Font(name="微软雅黑", size=11)

    def map_values(report_data):
        map_order_type = {
            'balance': '钱包',
            'baoyang': '保养',
            'coffee': '星巴克',
            'daijia': '代驾',
            'dianying': '电影',
            'huafei': '话费',
            'huoche': '火车',
            'jiadianqingxi': '家电清洗',
            'jiadianweixiu': '家电维修',
            'jiayouka': '加油卡',
            'jipiao': '机票',
            'jiudian': '酒店',
            'liuliang': '流量',
            'menpiao': '门票',
            'paotui': '跑腿',
            'piaowu': '票务',
            'sdm': '水电煤',
            'shenghuojiaofei': '生活缴费',
            'shengxian': '生鲜',
            'waimai': '外卖',
            'xianhua': '鲜花',
            'xihu': '洗护',
            'zhuanche': '专车',
            'weizhang': '违章缴费'
        }
        map_order_channel = {
            'ele': '饿了么',
            'tieyou': '铁友',
            'dhst': '大汉三通',
            'zhenlv': '真旅',
            'ofpay': '欧飞',
            'balance': '菠萝觅',
            'linqu': '邻趣',
            'e': 'e代驾',
            'fft': '付费通',
            'sfdj': '十分到家',
            'elong': '艺龙',
            'tidy': '泰迪',
            'ai': '爱代驾',
            'ctrip': '携程',
            'lvmama': '驴妈妈',
            'tongcheng': '同程',
            'gaoyang': '高阳',
            'qunar': '去哪儿',
            'qunar_baitour': '去哪儿-百拓',
            'yiguo': '易果',
            'zmn': '啄木鸟',
            'didi': '滴滴',
            'yidao': '易到',
            'dsby': '大师保养',
            'chinapay': '银联',
            'kou': '抠电影',
            'zzw': '蜘蛛网',
            'xishiqu': '西十区',
            'diandian': '典典养车',
            'carlife': '车生活'
        }
        map_pay_channel = {
            'wx': '微信',
            'alipay': '支付宝',
            'alipay_wap':'支付宝_wap',
            'balance': '菠萝觅',
            'chinapay': '银联',
            'ccb': '建行',
            'bosc': '上行',
            'icbc':'工行',
        }
        map_app_code = {
            'boluome': '菠萝觅',
            'fft': '付费通',
            'allinpay': '通联钱包',
            'jst': '聚事通',
            'roobo': 'Roobo智能生活',
            'chubao': '触宝电话',
            'jhdp': '聚浩大屏',
            '91jincai': '91金彩',
            'mybosc': '上行快线',
            '91ala': '51返呗',
            'scity': '市民服务'
        }

        report_data['appCode'] = map_app_code[report_data['appCode']]
        report_data['orderType'] = map_order_type[report_data['orderType']]

#         report_data['payChannel'] = ''
        pay_channel = report_data.get('payChannel', '')
        if ',' in pay_channel:
            report_data['payChannel'] = ''
            for key in pay_channel.split(','):
                report_data['payChannel'] += map_pay_channel[key] + ','
            report_data['payChannel'] = report_data['payChannel'][:-1]
        elif pay_channel:
            report_data['payChannel'] = map_pay_channel[pay_channel]

        report_data['orderChannel'] = map_order_channel[report_data['orderChannel'].lower()]
        return report_data

    def map_keys(report_data):
        map_key = {
            'appCode': '应用名称',
            'orderId': '订单号',
            'orderType': '品类',
            'orderChannel': '供应商',
            'partnerId': '供应商订单号',
            'orderPhone': '手机号',
            'userId': '用户ID',
            'orderName': '订单名称',
            'orderPrice': '订单金额',
            'payPrice': '支付金额',
            'deductionPrice': '优惠金额',
            'orderStatus': '订单状态',
            'payChannel': '支付方式',
            'orderTime': '下单时间',
            'paidTime': '支付时间',
            'payId': '支付订单号',
        }
        report_data = {
            map_key[k]: v
            for k, v in report_data.items()
            if k in map_key}
        return report_data

    for item in reports_data:
        item = map_values(item)
        item = map_keys(item)
#         pprint(item['支付金额'])
        row = ws.max_row + 1
        for col in range(1, ws.max_column + 1):
            ws.cell(row=row, column=col).value = item.get(ws.cell(row=1, column=col).value)
            ws.cell(row=row, column=7).value = "=E{0}-SUMPRODUCT((A:A=A{0})*1,F:F)".format(row)
    render_report(ws)
    wb.save(out_path)

    return

In [4]:
def prepare_transaction_data(mdb, rdb_1, rdb_2, app_code, start, end):
    """参数验证"""
    try:
        start = start.split('-')
        end = end.split('-')
        start_time = time.mktime((int(start[0]), int(start[1]), int(start[2]), 0, 0, 0, 0, 0, 0)) * 1000
        end_time = time.mktime((int(end[0]), int(end[1]), int(end[2]), 24, 0, 0, 0, 0, 0)) * 1000
    except ValueError as e:
        print('时间格式错误!')
        print('请输入如下格式的时间%Y-%m-%d，如2017-01-01')
        return

    if end_time > time.time() * 1000:
        return

    rst = mdb['order_lite_list'].find(
        {'appCode': app_code,
         'createdAt': {'$gte': start_time, '$lt': end_time}},
        projection={'_id': 0}
    )

    if not rst:
        return

    data = []
    for item in rst:
        order_type = item['orderType']
        order_id = item['id']
        order_detail = mdb['order_{}'.format(order_type)].find_one({'id': order_id})
        if not order_detail:
            continue
        order_channel = item['channel']
        order_phone = order_detail.get('phone')
        user_id = item['userId']
        order_name = item['name']
        partner_id = order_detail['partnerId']
        order_time = get_datetime_from_microsecond(item['createdAt'])
        order_status = item['displayStatus']
        status = item['status']
        order_price = order_detail['orderPrice']
        price = order_detail['price']
        deduction_price = float(order_price) - float(price)
        paid_list = list(set(order_detail.get('paidList', [])))
        paid_list = [v for v in paid_list if v]
        paid_list.sort()
#         print(paid_list)
        if not paid_list:
            paid_list = [order_id]
        else:
            print(paid_list)

        for paid_id in paid_list:
            data.append(
                {
                    'orderType': order_type,
                    'orderId': order_id,
                    'orderChannel': order_channel,
                    'orderPhone': order_phone,
                    'userId': user_id,
                    'orderName': order_name,
                    'orderTime': order_time,
                    'orderStatus': order_status,
                    'status': status,
                    'partnerId': partner_id,
                    'appCode': app_code,
                    'orderPrice': order_price,
                    'price': price,
                    'deductionPrice':  deduction_price,
                    'paidId': paid_id,
                }
            )

    report_data = []
    for item in data:
        # redis 数据库获取缺失订单价格数据字段，1为完成订单，2为退款订单
        payment_data = rdb_1.hgetall('p:{0}:{1}'.format(item['appCode'],
                                                        item['paidId']))
        refund_data = rdb_2.hgetall('r:{0}:{1}'.format(item['appCode'],
                                                       item['paidId']))

        status = item['status']
        pay_price = None
        
        if payment_data.get('status', '-1') in ['2', '3']:
            pay_id = payment_data.get('serialNum', '')
            pay_channel = payment_data.get('channel', '')

            paid_time = payment_data.get('paidAt')
            if paid_time:
                paid_time = get_datetime_from_microsecond(float(paid_time))

            pay_price = payment_data.get('payPrice')
            if pay_price:
                pay_price = float(pay_price) / 100

            item['payId'] = pay_id
            item['payChannel'] = pay_channel
            item['paidTime'] = paid_time
            item['payPrice'] = pay_price
            item['orderName'] = payment_data['name'] if payment_data.get('name') else item['orderName']

        elif status == 7:
            refunded_time = refund_data.get('createdAt')
            if refunded_time:
                refunded_time = get_datetime_from_microsecond(float(refunded_time))

            refunded_price = refund_data.get('refundedPrice')
            if refunded_price:
                refunded_price = float(refunded_price) / 100

            item['refundedTime'] = refunded_time
            item['refundedPrice'] = refunded_price

        report_data.append(item)

    return report_data
def get_datetime_from_microsecond(timestamp):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp / 1000))

In [5]:
if __name__=="__main__":
    app_code='boluome'
    start_date='2017-01-01'
    end_date='2017-03-22'
    reports_data = prepare_transaction_data(r.mdb,
                                                r.rdb_1,
                                                r.rdb_2,
                                                app_code,
                                                start_date,
                                                end_date)

#     pprint(reports_data)
    field_headers = ['订单号', '品类', '供应商', '订单名称', '订单金额', '支付金额',
                     '优惠金额', '订单状态', '支付方式', '下单时间', '支付时间', '支付订单号']

    generate_transaction_report(reports_data,
                                field_headers,
                                "交易明细报表.xlsx")


['10000039719200']
['10000039717501']
['10000039683900', '10000039683901']
['10000039683301']
['10000039681100']
['10000039668600']


In [5]:
r.rdb_1.hgetall('p:{0}:{1}'.format('boluome','100000397197'))

{}

In [6]:
r.rdb_1.hgetall('p:boluome:100000397197')

{}

In [7]:
a={'a':[ 
        "10000039719200", 
        "10000039719200",
    "10000039719201",
    "10000039719202"
    ]
}
b=(list(set(a.get('a',[])+['10000039719201'])))
b.sort()
b
# b=a.get('a',[])+['222']
# b

['10000039719200', '10000039719201', '10000039719202']